In [22]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carrega o dataset Iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)  # organizando em um DataFrame
df['target'] = iris.target  # rótulos

# Divide os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

# Treina o modelo
modelo = LogisticRegression(max_iter=200)
modelo.fit(X_train, y_train)  # treinando o modelo

# Função para analisar instâncias
def analisar_instancias(instancia_para_analisar=2):
    num_instancias = len(X_test)
    
    # Se nenhuma instância for especificada, analisa todas as instâncias
    instancias_para_analisar = range(num_instancias) if instancia_para_analisar is None else [instancia_para_analisar]
    
    # Loop para analisar instâncias
    for idx in instancias_para_analisar:
        Vs = X_test.iloc[idx].to_dict()  # Utilizando a instância de teste especificada
        instancia_test = X_test.iloc[[idx]]  # Mantém como DataFrame para preservar os nomes das características

        # Previsão de probabilidades para a instância
        probs = modelo.predict_proba(instancia_test)[0]  # Obtém as probabilidades para cada classe

        # O valor de gamma_A é a probabilidade da classe verdadeira
        classe_verdadeira = y_test.iloc[idx]  # Obtém a classe verdadeira da instância
        gamma_A = probs[classe_verdadeira]  # Extrai a probabilidade correspondente à classe verdadeira
        
        # Cálculo do valor delta para cada feature
        delta = []
        w = modelo.coef_[0]  # pesos do modelo treinado

        # Calcula o delta para cada feature
        for i, feature in enumerate(df.columns[:-1]):
            if w[i] < 0:
                delta.append((Vs[feature] - df[feature].max()) * w[i])
                print((Vs[feature] - df[feature].max()) * w[i])
                print(f'{df[feature].min()} - {Vs[feature]} *{w[i]}')
                #print(w[i])
            else:
                delta.append((df[feature].min() - Vs[feature]) * w[i])

        # Calcula R como a soma dos deltas menos gamma_A
        R = sum(delta) - gamma_A  # Atualiza R para incluir gamma_A
        
        # Computa a PI-explicação para a instância atual
        Xpl = one_explanation(Vs, delta, R)

        # Imprime os resultados
        print(f"\nInstância {idx}:")
        print(f"Classe verdadeira: {classe_verdadeira} ({iris.target_names[classe_verdadeira]})")
        print(f"Probabilidades: {probs}")
        print(f"Valor de gamma_A: {gamma_A}")
        print(f"PI-Explicação: ")
        for item in Xpl:
            print(f"- {item}")

# Função para calcular a PI-explicação
def one_explanation(Vs, delta, R):
    Xpl = []  # Inicializa a lista de PI-explicação
    # Ordena o delta junto com seus índices, em ordem decrescente de valor absoluto
    delta_sorted = sorted(enumerate(delta), key=lambda x: abs(x[1]), reverse=True)  # enumerando os valores de delta em tuplas e ordenando
    R_atual = R  # Inicializa o limiar atual
    Idx = 0  # Inicializa o índice para iterar
    
    while R_atual >= 0 and Idx < len(delta_sorted):
        sorted_idx, delta_value = delta_sorted[Idx]  # Desempacota o índice e o valor do delta
        feature = X_test.columns[sorted_idx]  # Obtém o nome da feature correspondente
        feature_value = Vs[feature]  # Obtém o valor da feature na instância Vs
        
        # Adiciona à explicação
        Xpl.append(f"{feature} - {feature_value}")  # Adiciona feature e valor à explicação
        
        R_atual -= delta_value  # Atualiza o limiar atual para manter ou parar o loop
        Idx += 1  # Incrementa o índice
    
    return Xpl  # Retorna a PI-explicação

# Exemplo de uso:
# Para analisar uma instância específica, passe o índice dela, como 0, 1, 2, etc.
# Exemplo: analisar_instancias(0)  -> analisa a instância 0
# Para analisar todas as instâncias, apenas execute a função sem argumento
analisar_instancias()  # Analisa todas as instâncias
# Ou para uma específica
# analisar_instancias(2)  # Analisa a instância 2


0.07876280643270722
4.3 - 7.7 *-0.3938140321635358
-0.0
1.0 - 6.9 *-2.375192750974152
0.1997454612360671
0.1 - 2.3 *-0.9987273061803346

Instância 2:
Classe verdadeira: 2 (virginica)
Probabilidades: [8.84412614e-09 1.54875125e-03 9.98451240e-01]
Valor de gamma_A: 0.9984512399032031
PI-Explicação: 


CODIGO PARTICIONADO

In [91]:
#Usando este código
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carrega o dataset Iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)  # organizando em um DataFrame
df['target'] = iris.target  # rótulos

# Divide os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

# Treina o modelo
modelo = LogisticRegression(max_iter=200)
modelo.fit(X_train, y_train)  # treinando o modelo

# Função para analisar instâncias
def analisar_instancias(instancia_para_analisar=1):
    num_instancias = len(X_test)
    
    # Se nenhuma instância for especificada, analisa todas as instâncias
    instancias_para_analisar = range(num_instancias) if instancia_para_analisar is None else [instancia_para_analisar]
    
    # Loop para analisar instâncias
    for idx in instancias_para_analisar:
        Vs = X_test.iloc[idx].to_dict()  # Utilizando a instância de teste especificada
        instancia_test = X_test.iloc[[idx]]  # Mantém como DataFrame para preservar os nomes das características

        # Previsão de probabilidades para a instância
        probs = modelo.predict_proba(instancia_test)[0]  # Obtém as probabilidades para cada classe

        # O valor de gamma_A é a probabilidade da classe verdadeira
        classe_verdadeira = y_test.iloc[idx]  # Obtém a classe verdadeira da instância
        gamma_A = probs[classe_verdadeira]  # Extrai a probabilidade correspondente à classe verdadeira
        
        # Cálculo do valor delta para cada feature
        delta = []
        w = modelo.coef_[0]  # pesos do modelo treinado

        # Calcula o delta para cada feature
        for i, feature in enumerate(df.columns[:-1]):
            if w[i] < 0:
                delta.append((Vs[feature] - df[feature].max()) * w[i])
            else:
                delta.append((df[feature].min() - Vs[feature]) * w[i])

        # Calcula R como a soma dos deltas menos gamma_A
        R = sum(delta) - gamma_A  # Atualiza R para incluir gamma_A
        # Computa a PI-explicação para a instância atual
        Xpl = one_explanation(Vs, delta, R)

        # Imprime os resultados
    #    print(f"\nInstância {idx}:")
    #    print(f"Classe verdadeira: {classe_verdadeira} ({iris.target_names[classe_verdadeira]})")
    #    print(f"Probabilidades: {probs}")
    #    print(f"Valor de gamma_A: {gamma_A}")
        print(f"PI-Explicação: ")
        for item in Xpl:
            print(f"- {item}")
        if not Xpl:
            print("Nenhuma PI-explicação encontrada para esta instância.\n")
# Função para calcular a PI-explicação
def one_explanation(Vs, delta, R):
    Xpl = []  # Inicializa a lista de PI-explicação
    
    # Ordena o delta em ordem decrescente de valor absoluto
    delta_sorted = sorted(delta, reverse=True)  # enumerando os valores de delta em tuplas e ordenando
    R_atual = R  # Inicializa o limiar atual
    Idx = 0  # Inicializa o índice para iterar
    
    while R_atual >= 0 and Idx < len(delta_sorted):
        delta_value = delta_sorted[Idx]  # Desempacota o índice e o valor do delta
        feature = X_test.columns[Idx]  # Obtém o nome da feature correspondente
        feature_value = Vs[feature]  # Obtém o valor da feature na instância Vs
        
        # Adiciona à explicação
        Xpl.append(f"{feature} - {feature_value}")  # Adiciona feature e valor à explicação
        
        R_atual -= delta_value  # Atualiza o limiar atual para manter ou parar o loop
        Idx += 1  # Incrementa o índice
    
    return Xpl  # Retorna a PI-explicação

analisar_instancias(None)  # Analisa todas as instâncias

PI-Explicação: 
- sepal length (cm) - 6.1
- sepal width (cm) - 2.8
PI-Explicação: 
- sepal length (cm) - 5.7
- sepal width (cm) - 3.8
PI-Explicação: 
Nenhuma PI-explicação encontrada para esta instância.

PI-Explicação: 
- sepal length (cm) - 6.0
- sepal width (cm) - 2.9
PI-Explicação: 
- sepal length (cm) - 6.8
PI-Explicação: 
- sepal length (cm) - 5.4
- sepal width (cm) - 3.4
PI-Explicação: 
- sepal length (cm) - 5.6
- sepal width (cm) - 2.9
PI-Explicação: 
- sepal length (cm) - 6.9
PI-Explicação: 
- sepal length (cm) - 6.2
- sepal width (cm) - 2.2
PI-Explicação: 
- sepal length (cm) - 5.8
- sepal width (cm) - 2.7
PI-Explicação: 
- sepal length (cm) - 6.5
PI-Explicação: 
- sepal length (cm) - 4.8
- sepal width (cm) - 3.0
PI-Explicação: 
- sepal length (cm) - 5.5
- sepal width (cm) - 3.5
PI-Explicação: 
- sepal length (cm) - 4.9
- sepal width (cm) - 3.1
PI-Explicação: 
- sepal length (cm) - 5.1
- sepal width (cm) - 3.8
PI-Explicação: 
- sepal length (cm) - 6.3
PI-Explicação: 
- sepal 

In [85]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carrega o dataset Iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)  # organizando em um DataFrame
df['target'] = iris.target  # rótulos

# Divide os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

# Treina o modelo
modelo = LogisticRegression(max_iter=200)
modelo.fit(X_train, y_train)  # treinando o modelo

# Usa a primeira instância dos dados de teste para a explicação
Vs = X_test.iloc[0].to_dict()  # utilizando a primeira instância dos dados de teste 
instancia_test = X_test.iloc[[0]]  # Mantem como DataFrame para preservar os nomes das características



# Previsão de probabilidades para a instância
# o predict_proba calcula a probabilidade de uma instância pertencer a uma das classes possíveis usando um modelo de classificação treinado.
probs = modelo.predict_proba(instancia_test)[0]  # Obtém as probabilidades para cada classe

# O valor de gamma_A é a probabilidade da classe verdadeira
classe_verdadeira = y_test.iloc[0]  # Obtém a classe verdadeira da instância
gamma_A = probs[classe_verdadeira]  # Extrai a probabilidade correspondente à classe verdadeira
print(classe_verdadeira)
print(gamma_A)
# Cálculo do valor delta para cada feature
delta = []
w = modelo.coef_[0]  # pesos do modelo treinado

# Calcula o delta para cada feature
for i, feature in enumerate(df.columns[:-1]):
    if w[i] < 0:
        delta.append((Vs[feature] - df[feature].max()) * w[i])
    else:
        delta.append((Vs[feature] - df[feature].min()) * w[i])

# Calcula R como a soma dos deltas menos gamma_A
R = sum(delta) - gamma_A  # Atualiza R para incluir gamma_A

def one_explanation(Vs, delta, R):
    Xpl = []  # Inicializa a lista de PI-explicação
    
    # Ordena o delta em ordem decrescente de valor absoluto
    delta_sorted = sorted(delta, reverse=True)  # enumerando os valores de delta em tuplas e ordenando
    R_atual = R  # Inicializa o limiar atual
    Idx = 0  # Inicializa o índice para iterar
    
    while R_atual >= 0 and Idx < len(delta_sorted):
        delta_value = delta_sorted[Idx]  # Desempacota o índice e o valor do delta
        feature = X_test.columns[Idx]  # Obtém o nome da feature correspondente
        feature_value = Vs[feature]  # Obtém o valor da feature na instância Vs
        
        # Adiciona à explicação
        Xpl.append(f"{feature} - {feature_value}")  # Adiciona feature e valor à explicação
        
        R_atual -= delta_value  # Atualiza o limiar atual para manter ou parar o loop
        Idx += 1  # Incrementa o índice
    
    return Xpl  # Retorna a PI-explicação

# Computa a PI-explicação
Xpl = one_explanation(Vs, delta, R)

# Imprime os resultados
print(f"Probabilidades: {probs}")
print(f"Classe verdadeira: {classe_verdadeira}")
print(f"Valor de gamma_A: {gamma_A}")
print(f"PI-Explicação: ")
for item in Xpl:
    print(f"- {item}")

1
0.8277146135745481
Probabilidades: [0.00380009 0.82771461 0.16848529]
Classe verdadeira: 1
Valor de gamma_A: 0.8277146135745481
PI-Explicação: 
- sepal length (cm) - 6.1
- sepal width (cm) - 2.8
- petal length (cm) - 4.7


In [2]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carrega o dataset Iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)  # Organizando em um DataFrame
df['target'] = iris.target  # Rótulos

# Divide os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

# Treina o modelo
modelo = LogisticRegression(max_iter=200)
modelo.fit(X_train, y_train)  # Treinando o modelo

# Função para analisar instâncias
def analisar_instancias(instancia_para_analisar=None):
    num_instancias = len(X_test)
    
    # Se nenhuma instância for especificada, analisa todas as instâncias
    instancias_para_analisar = range(num_instancias) if instancia_para_analisar is None else [instancia_para_analisar]
    
    # Loop para analisar instâncias
    for idx in instancias_para_analisar:
        Vs = X_test.iloc[idx].to_dict()  # Utilizando a instância de teste especificada
        instancia_test = X_test.iloc[[idx]]  # Mantém como DataFrame para preservar os nomes das características

        # Previsão de probabilidades para a instância
        probs = modelo.predict_proba(instancia_test)[0]  # Obtém as probabilidades para cada classe

        # O valor de gamma_A é a probabilidade da classe verdadeira
        classe_verdadeira = y_test.iloc[idx]  # Obtém a classe verdadeira da instância
        gamma_A = probs[classe_verdadeira]  # Extrai a probabilidade correspondente à classe verdadeira
        
        # Cálculo do valor delta para cada feature
        delta = []
        w = modelo.coef_[0]  # Pesos do modelo treinado

        # Calcula o delta para cada feature
        for i, feature in enumerate(df.columns[:-1]):
            if w[i] < 0:
                delta_val = (Vs[feature] - df[feature].max()) * w[i]
            else:
                delta_val = (df[feature].min() - Vs[feature]) * w[i]
                
            delta.append(delta_val)
            print(f"Delta -> {feature} -> {delta_val}")
            print(f'Vs -> {Vs[feature]}')
            print(f' df -> {df[feature].max()}-peso> {w[i]}')
        # Calcula R como a soma dos deltas menos gamma_A
        R = sum(delta) - gamma_A
        print(f"Valor de R para a instância {idx}: {R}")
        print(delta)
        print(f"valores -> soma dos delta + gamma_A -> {abs(sum(delta))}: {gamma_A}")
        # Computa a PI-explicação para a instância atual
        Xpl = one_explanation(Vs, delta, R)

        # Imprime os resultados
        print(f"\nInstância {idx}:")
        print(f"Classe verdadeira: {classe_verdadeira} ({iris.target_names[classe_verdadeira]})")
        print(f"Probabilidades: {probs}")
        print(f"Valor de gamma_A: {gamma_A}")
        print(f"PI-Explicação: ")
        for item in Xpl:
            print(f"- {item}")
        sem_explicacao = []
        if not Xpl:
            sem_explicacao.append(idx)
            print("Nenhuma PI-explicação encontrada para esta instância.\n")
        
# Função para calcular a PI-explicação
def one_explanation(Vs, delta, R):
    Xpl = []  # Inicializa a lista de PI-explicação
    # Ordena o delta junto com seus índices, em ordem decrescente de valor absoluto
    delta_sorted = sorted(enumerate(delta), key=lambda x: x[1], reverse=True)
    R_atual = R  # Inicializa o limiar atual
    Idx = 0  # Inicializa o índice para iterar
    
    while R_atual >= 0 and Idx < len(delta_sorted):
        sorted_idx, delta_value = delta_sorted[Idx]  # Desempacota o índice e o valor do delta
        feature = X_test.columns[sorted_idx]  # Obtém o nome da feature correspondente
        feature_value = Vs[feature]  # Obtém o valor da feature na instância Vs
        
        # Adiciona à explicação
        Xpl.append(f"{feature} - {feature_value}")  # Adiciona feature e valor à explicação
        
        R_atual -= delta_value  # Atualiza o limiar atual para manter ou parar o loop
        Idx += 1  # Incrementa o índice
    
    return Xpl  # Retorna a PI-explicação

# Executa a análise na instância 2 para verificar o problema
analisar_instancias(2)  # Analisa a instância 2


Delta -> sepal length (cm) -> 0.07876280643270722
Vs -> 7.7
 df -> 7.9-peso> -0.3938140321635358
Delta -> sepal width (cm) -> -0.5773233879333367
Vs -> 2.6
 df -> 4.4-peso> 0.962205646555561
Delta -> petal length (cm) -> -0.0
Vs -> 6.9
 df -> 6.9-peso> -2.375192750974152
Delta -> petal width (cm) -> 0.1997454612360671
Vs -> 2.3
 df -> 2.5-peso> -0.9987273061803346
Valor de R para a instância 2: -1.2972663601677654
[0.07876280643270722, -0.5773233879333367, -0.0, 0.1997454612360671]
valores -> soma dos delta + gamma_A -> 0.29881512026456236: 0.9984512399032031

Instância 2:
Classe verdadeira: 2 (virginica)
Probabilidades: [8.84412614e-09 1.54875125e-03 9.98451240e-01]
Valor de gamma_A: 0.9984512399032031
PI-Explicação: 
Nenhuma PI-explicação encontrada para esta instância.



In [81]:
# Exibe os valores das características da instância 2
print("Valores da Instância 2:")
print(X_test.iloc[24])  # Acessa a linha 2 (índice 2) em X_test


Valores da Instância 2:
sepal length (cm)    7.9
sepal width (cm)     3.8
petal length (cm)    6.4
petal width (cm)     2.0
Name: 131, dtype: float64


In [80]:
# Lista para armazenar os índices das instâncias sem PI-explicação
instancias_sem_explicacao = []

# Itera sobre cada instância dos dados de teste
for idx in range(len(X_test)):
    Vs = X_test.iloc[[idx]].to_dict('records')[0]  # Obtendo cada instancia como um dicionario
    instancia_test = X_test.iloc[[idx]]  # Mantem como DataFrame para preservar os nomes das características

    # Previsão de probabilidades para a instância
    probs = modelo.predict_proba(instancia_test)[0]  # Obtém as probabilidades para cada classe

    # O valor de gamma_A é a probabilidade da classe verdadeira
    classe_verdadeira = y_test.iloc[idx]  # Obtém a classe verdadeira da instância
    gamma_A = probs[classe_verdadeira]  # Extrai a probabilidade correspondente à classe verdadeira

    # Calcula o delta para cada feature
    delta = []
    for i, feature in enumerate(df.columns[:-1]):
        if w[i] < 0:
            delta.append((Vs[feature] - df[feature].max()) * w[i])
        else:
            delta.append((df[feature].min() - Vs[feature]) * w[i])

    # Calcula R como a soma dos deltas menos gamma_A
    R = sum(delta) - gamma_A  # Atualiza R para incluir gamma_A

    # Computa a PI-explicação
    Xpl = one_explanation(Vs, delta, R)

    # Verifica se a PI-explicação está vazia
    if not Xpl:
        instancias_sem_explicacao.append(idx)  # Adiciona o índice da instância à lista

# Imprime a lista de instâncias sem PI-explicação
print(f"Instâncias sem PI-explicação: {instancias_sem_explicacao}")

Instâncias sem PI-explicação: [2, 24]


In [6]:
# Carrega o dataset Iris completo
X = df.drop('target', axis=1)  # Utilizando todas as características
y = df['target']  # Utilizando todos os rótulos

# Treina o modelo usando o dataset completo
modelo = LogisticRegression(max_iter=200)
modelo.fit(X, y)  # Treinando o modelo com todas as instâncias

# Modifique a função `analisar_instancias` para utilizar o dataset completo
def analisar_instancias(instancia_para_analisar=None):
    num_instancias = len(X)

    # Se nenhuma instância for especificada, analisa todas as instâncias
    instancias_para_analisar = range(num_instancias) if instancia_para_analisar is None else [instancia_para_analisar]

    # Loop para analisar instâncias
    for idx in instancias_para_analisar:
        Vs = X.iloc[idx].to_dict()  # Utilizando a instância especificada
        instancia_test = X.iloc[[idx]]  # Mantém como DataFrame para preservar os nomes das características

        # Previsão de probabilidades para a instância
        probs = modelo.predict_proba(instancia_test)[0]  # Obtém as probabilidades para cada classe

        # O valor de gamma_A é a probabilidade da classe verdadeira
        classe_verdadeira = y.iloc[idx]  # Obtém a classe verdadeira da instância
        gamma_A = probs[classe_verdadeira]  # Extrai a probabilidade correspondente à classe verdadeira

        # Cálculo do valor delta para cada feature
        delta = []
        w = modelo.coef_[0]  # Pesos do modelo treinado

        # Calcula o delta para cada feature
        for i, feature in enumerate(df.columns[:-1]):
            if w[i] < 0:
                delta.append((Vs[feature] - df[feature].max()) * w[i])
            else:
                delta.append((Vs[feature] - df[feature].min()) * w[i])

        # Calcula R como a soma dos deltas menos gamma_A
        R = sum(delta) - gamma_A
        Xpl = one_explanation(Vs, delta, R)

        # Imprime os resultados
        print(f"\nInstância {idx}:")
        print(f"Classe: {classe_verdadeira} -> {iris.target_names[classe_verdadeira]}")
        print(f"PI-Explicação: ")
        for item in Xpl:
            print(f"- {item}")
        if not Xpl:
            print('_No-PI-explanation_'*3)

# Chamada da função para analisar todas as instâncias
analisar_instancias(2)  # Analisa todas as 150 instâncias



Instância 2:
Classe: 0 -> setosa
PI-Explicação: 
- petal length (cm) - 1.3
- petal width (cm) - 0.2
- sepal length (cm) - 4.7
- sepal width (cm) - 3.2
